# Notebook for importing from the Unihan Database

In [1]:
# Assumes the Unihan database has been downloaded from http://www.unicode.org/Public/UCD/latest/ucd/Unihan.zip
# and placed in the directory ../Unihan
import sys

import pandas as pd
import numpy as np

# Load Unihan Readings table
colnames = ['codepoint', 'fieldname', 'value']
types = {'codepoint': np.string_, 'fieldname': np.string_, 'value': np.string_}
readings = pd.read_table('../Unihan/Unihan_Readings.txt', names=colnames, dtype=types)
readings.set_index("codepoint")
print readings['codepoint'].count() # Number of rows

# Load Unihan Variants table
colnames = ['codepoint', 'fieldname', 'value']
types = {'codepoint': np.string_, 'fieldname': np.string_, 'value': np.string_}
variants = pd.read_table('../Unihan/Unihan_Variants.txt', names=colnames, dtype=types)
print variants['codepoint'].count() # Number of rows

readings.tail()

187561
12375


,codepoint,fieldname,value
187556,U+2F994,kCantonese,fong1
187557,U+2F9B2,kCantonese,kwai4
187558,U+2F9BC,kCantonese,sip3
187559,U+2F9D4,kCantonese,gun3 gwun3
187560,# EOF,NaN,NaN


In [2]:
chinese = u'𦥶'
cp = 'U+' + hex(ord(chinese)).replace('0x', '').upper()
chineseDf = readings[readings.codepoint == cp]
chineseDf

TypeError: ord() expected a character, but string of length 2 found

In [3]:
variantsDf = variants[variants.codepoint == cp]
variantsDf

NameError: name 'cp' is not defined

In [4]:
english = "\\N"
englishDF = chineseDf[chineseDf.fieldname == 'kDefinition']['value']
if len(englishDF) > 0:
    english = englishDF.iloc[0]
print english

pinyin = chineseDf[chineseDf.fieldname == 'kMandarin']['value'].iloc[0].decode('utf-8')
print pinyin

# Kinds of variants
notes = ""
c = ""
value = variantsDf[variantsDf.fieldname == 'kSemanticVariant']['value']
if len(value) > 0:
    val = value.iloc[0].split('<')[0]
    val = val.replace("U+", "")
    c = unichr(int(val, 16))
    notes = "Semantic variant: %s " % c
print notes

simplified = chinese
traditional = "\\N"
value = variantsDf[variantsDf.fieldname == 'kSimplifiedVariant']['value']
if len(value) > 0:
    val = value.iloc[0].split('<')[0]
    val = val.replace("U+", "")
    simplified = unichr(int(val, 16))
    traditional = chinese
print simplified

grammar = "\\N"
if english != "\\N":
    grammar = "noun"
print grammar

luid = 45706
notes = "%s (Unihan '%s')" % (notes, chinese)
domain = u"古文\tClassical Chinese"
print u"%d\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t\\N\t\\N\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, traditional, pinyin, english, grammar, domain, notes, luid)

NameError: name 'chineseDf' is not defined

In [5]:
# Generate lexical entries for the characters in file unknown.txt
luid = 57036
colnames = ['codepoint', 'char']
types = {'char': np.string_, 'codepoint': np.string_}
unknownDF = pd.read_table('unknown.txt', names = colnames, dtype=types, header = None)
unknownDF

,codepoint,char
0,U+26976,𦥶
1,U+48DC,䣜
2,U+687A,桺
3,U+49B2,䦲
4,U+4959,䥙
5,U+8045,聅
6,U+5945,奅
7,U+25206,𥈆
8,U+48C4,䣄
9,U+8C3D,谽


In [6]:
unknownReadings = pd.merge(unknownDF, readings, on = "codepoint")
unknownReadings.set_index("codepoint")
unknownReadings

,codepoint,char,fieldname,value
0,U+48DC,䣜,kDefinition,(same as 酇) name of a county (in today's Henan...
1,U+48DC,䣜,kHanyuPinyin,63792.050:cuó
2,U+48DC,䣜,kMandarin,cuó
3,U+687A,桺,kJapaneseKun,YANAGI
4,U+687A,桺,kJapaneseOn,RYUU
5,U+687A,桺,kMandarin,liǔ
6,U+49B2,䦲,kCantonese,zim4
7,U+49B2,䦲,kDefinition,eaves of a ancestral temple (of the ruling fam...
8,U+49B2,䦲,kHanyuPinyin,"74320.080:yán,qiàn,chàn"
9,U+49B2,䦲,kMandarin,yán


In [7]:
pivoted = unknownReadings.pivot("codepoint", "fieldname", "value")
del pivoted["kCantonese"]
del pivoted["kHanyuPinyin"]
del pivoted["kJapaneseKun"]
del pivoted["kJapaneseOn"]
del pivoted["kVietnamese"]
del pivoted["kXHC1983"]
pivoted = pivoted.rename(columns = {"kDefinition":"english", "kMandarin":"pinyin"})
pivoted

fieldname,english,kHangul,kHanyuPinlu,kKorean,pinyin,kTang
codepoint,,,,,,
U+20000,the sound made by breathing in; oh! (cf. U+311...,None,None,None,hē,None
U+20112,None,None,None,None,chuí,None
U+202A9,None,None,None,None,zhuàn,None
U+203EE,None,None,None,None,duì,None
U+2043F,None,None,None,None,hōng,None
U+204D7,None,None,None,None,fù,None
U+204DC,None,None,None,None,liǎng,None
U+20509,a roll,None,None,None,juàn,None
U+2052F,None,None,None,None,bān,None


In [8]:
pivoted = pivoted.fillna("\\N")
pivoted

fieldname,english,kHangul,kHanyuPinlu,kKorean,pinyin,kTang
codepoint,,,,,,
U+20000,the sound made by breathing in; oh! (cf. U+311...,\N,\N,\N,hē,\N
U+20112,\N,\N,\N,\N,chuí,\N
U+202A9,\N,\N,\N,\N,zhuàn,\N
U+203EE,\N,\N,\N,\N,duì,\N
U+2043F,\N,\N,\N,\N,hōng,\N
U+204D7,\N,\N,\N,\N,fù,\N
U+204DC,\N,\N,\N,\N,liǎng,\N
U+20509,a roll,\N,\N,\N,juàn,\N
U+2052F,\N,\N,\N,\N,bān,\N


In [9]:
unknownDF = unknownDF.join(pivoted, on = "codepoint", lsuffix = "_")
unknownDF

,codepoint,char,english,kHangul,kHanyuPinlu,kKorean,pinyin,kTang
0,U+26976,𦥶,NaN,NaN,NaN,NaN,NaN,NaN
1,U+48DC,䣜,(same as 酇) name of a county (in today's Henan...,\N,\N,\N,cuó,\N
2,U+687A,桺,\N,\N,\N,\N,liǔ,\N
3,U+49B2,䦲,eaves of a ancestral temple (of the ruling fam...,\N,\N,\N,yán,\N
4,U+4959,䥙,"(same as 鐆) a speculum used in ancient times, ...",\N,\N,\N,suì,\N
5,U+8045,聅,\N,\N,\N,\N,chè,\N
6,U+5945,奅,\N,\N,\N,\N,pào,\N
7,U+25206,𥈆,\N,\N,\N,\N,mào,\N
8,U+48C4,䣄,name of a place in today's Shandong Province,\N,\N,\N,tú,\N
9,U+8C3D,谽,a mouth or opening,\N,\N,\N,hān,\N


In [10]:
unknownDF['id'] = range(luid, luid + len(unknownDF))
unknownDF['traditional'] = "\\N"
getGrammar = lambda x: "noun" if x.english != "\\N" else "\\N"
unknownDF['grammar'] = unknownDF.apply(getGrammar, axis = 1)
unknownDF['concept_cn'] = "\\N"
unknownDF['concept_en'] = "\\N"
unknownDF['domain_cn'] = "古文"
unknownDF['domain_en'] = "Classical Chinese"
unknownDF['subdomain_cn'] = "\\N"
unknownDF['subdomain_en'] = "\\N"
unknownDF['mp3'] = "\\N"
unknownDF['image'] = "\\N"
writeNotes = lambda c: "(Unihan '" + c + "')"
unknownDF['notes'] = unknownDF['char'].map(writeNotes)
unknownDF['headword'] = unknownDF['id']
unknownDF

,codepoint,char,english,kHangul,kHanyuPinlu,kKorean,pinyin,kTang,id,traditional,...,concept_cn,concept_en,domain_cn,domain_en,subdomain_cn,subdomain_en,mp3,image,notes,headword
0,U+26976,𦥶,NaN,NaN,NaN,NaN,NaN,NaN,57036,\N,...,\N,\N,古文,Classical Chinese,\N,\N,\N,\N,(Unihan '𦥶'),57036
1,U+48DC,䣜,(same as 酇) name of a county (in today's Henan...,\N,\N,\N,cuó,\N,57037,\N,...,\N,\N,古文,Classical Chinese,\N,\N,\N,\N,(Unihan '䣜'),57037
2,U+687A,桺,\N,\N,\N,\N,liǔ,\N,57038,\N,...,\N,\N,古文,Classical Chinese,\N,\N,\N,\N,(Unihan '桺'),57038
3,U+49B2,䦲,eaves of a ancestral temple (of the ruling fam...,\N,\N,\N,yán,\N,57039,\N,...,\N,\N,古文,Classical Chinese,\N,\N,\N,\N,(Unihan '䦲'),57039
4,U+4959,䥙,"(same as 鐆) a speculum used in ancient times, ...",\N,\N,\N,suì,\N,57040,\N,...,\N,\N,古文,Classical Chinese,\N,\N,\N,\N,(Unihan '䥙'),57040
5,U+8045,聅,\N,\N,\N,\N,chè,\N,57041,\N,...,\N,\N,古文,Classical Chinese,\N,\N,\N,\N,(Unihan '聅'),57041
6,U+5945,奅,\N,\N,\N,\N,pào,\N,57042,\N,...,\N,\N,古文,Classical Chinese,\N,\N,\N,\N,(Unihan '奅'),57042
7,U+25206,𥈆,\N,\N,\N,\N,mào,\N,57043,\N,...,\N,\N,古文,Classical Chinese,\N,\N,\N,\N,(Unihan '𥈆'),57043
8,U+48C4,䣄,name of a place in today's Shandong Province,\N,\N,\N,tú,\N,57044,\N,...,\N,\N,古文,Classical Chinese,\N,\N,\N,\N,(Unihan '䣄'),57044
9,U+8C3D,谽,a mouth or opening,\N,\N,\N,hān,\N,57045,\N,...,\N,\N,古文,Classical Chinese,\N,\N,\N,\N,(Unihan '谽'),57045


In [11]:
cols = ['id', 'char', 'traditional', 'pinyin', 'english', 'grammar', 'concept_cn', 'concept_en', 'domain_cn', 'domain_en', 'subdomain_cn', 'subdomain_en', 'mp3', 'image', 'notes', 'headword']
unknownDF = unknownDF[cols]
unknownDF.to_csv(sys.stdout, index = False, header = False, sep = "\t", cols = cols)

57036	𦥶	\N			noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '𦥶')	57036
57037	䣜	\N	cuó	(same as 酇) name of a county (in today's Henan Province) in ancient times	noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '䣜')	57037
57038	桺	\N	liǔ	\N	\N	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '桺')	57038
57039	䦲	\N	yán	eaves of a ancestral temple (of the ruling family), door of a shrine or a temple, small opening (of a door), to spy; to peep	noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '䦲')	57039
57040	䥙	\N	suì	(same as 鐆) a speculum used in ancient times, to produce fire from the rays of the sun	noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '䥙')	57040
57041	聅	\N	chè	\N	\N	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '聅')	57041
57042	奅	\N	pào	\N	\N	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '奅')	57042
57043	𥈆	\N	mào	\N	\N	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '𥈆')	57043
57044	䣄	\N	tú	name of a place in today's Shandong Province	noun	\N	\N	古文	Classical Chine